In [1]:
from lxml import etree
import pandas as pd
from collections import Counter
import os
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from numpy import array
import numpy as np

import subprocess


In [5]:
from tokenizers import Tokenizer


In [7]:
import torch

In [2]:
BIG_FILE_URL = 'https://raw.githubusercontent.com/dscape/spell/master/test/resources/big.txt'

# Let's download the file and save it somewhere
from requests import get
with open('big.txt', 'wb') as big_f:
    response = get(BIG_FILE_URL, )
    
    if response.status_code == 200:
        big_f.write(response.content)
    else:
        print("Unable to get the file: {}".format(response.reason))

In [7]:
# For the user's convenience `tokenizers` provides some very high-level classes encapsulating
# the overall pipeline for various well-known tokenization algorithm. 
# Everything described below can be replaced by the ByteLevelBPETokenizer class. 

from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence, BertNormalizer
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
tokenizer = Tokenizer(BPE())

# Then we enable lower-casing and unicode-normalization
# The Sequence normalizer allows us to combine multiple Normalizer that will be
# executed in order.
tokenizer.normalizer = Sequence([
    BertNormalizer(),
    Lowercase()
])

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
tokenizer.decoder = ByteLevelDecoder()

In [8]:
from tokenizers.trainers import BpeTrainer

# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size=25000, show_progress=True, initial_alphabet=ByteLevel.alphabet())
tokenizer.train(files=["big.txt"], trainer=trainer)

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 25000


In [9]:
# You will see the generated files in the output.
tokenizer.model.save('.')

['.\\vocab.json', '.\\merges.txt']

In [10]:
# Let's tokenizer a simple input
tokenizer.model = BPE('vocab.json', 'merges.txt')
encoding = tokenizer.encode("This is a simple input to be tokenized")

print("Encoded string: {}".format(encoding.tokens))

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded string: ['Ġthis', 'Ġis', 'Ġa', 'Ġsimple', 'Ġin', 'put', 'Ġto', 'Ġbe', 'Ġtoken', 'ized']
Decoded string:  this is a simple input to be tokenized
C:\Users\calvotello\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Deprecated in 0.9.0: BPE.__init__ will not create from files anymore, try `BPE.from_file` instead
  


In [17]:
import pytorch

ModuleNotFoundError: No module named 'pytorch'

In [8]:
from transformers import BertTokenizer

In [19]:
BertTokenizer.from_pretrained?

Signature:
BertTokenizer.from_pretrained(
    pretrained_model_name_or_path: Union[str, os.PathLike],
    *init_inputs,
    **kwargs,
)
Docstring:
Instantiate a :class:`~transformers.tokenization_utils_base.PreTrainedTokenizerBase` (or a derived class) from
a predefined tokenizer.

Args:
    pretrained_model_name_or_path (:obj:`str` or :obj:`os.PathLike`):
        Can be either:

        - A string, the `model id` of a predefined tokenizer hosted inside a model repo on huggingface.co.
          Valid model ids can be located at the root-level, like ``bert-base-uncased``, or namespaced under a
          user or organization name, like ``dbmdz/bert-base-german-cased``.
        - A path to a `directory` containing vocabulary files required by the tokenizer, for instance saved
          using the :meth:`~transformers.tokenization_utils_base.PreTrainedTokenizerBase.save_pretrained`
          method, e.g., ``./my_model_directory/``.
        - (**Deprecated**, not applicable to all derived cl

In [26]:
#tz = BertTokenizer.from_pretrained("bert-base-uncased")
tz = BertTokenizer.from_pretrained("bert-base-cased")



In [23]:
sent = "Let's learn deep learning!"

In [24]:
# Encode the sentence
encoded = tz.encode_plus(
    text=sent,  # the sentence to be encoded
    add_special_tokens=True,  # Add [CLS] and [SEP]
    max_length = 64,  # maximum length of a sentence
    pad_to_max_length=True,  # Add [PAD]s
    return_attention_mask = True,  # Generate the attention mask
    return_tensors = 'np',  # ask the function to return PyTorch tensors
)

# Get the input IDs and attention mask in tensor format
input_ids = encoded['input_ids']
attn_mask = encoded['attention_mask']

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\calvotello\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [25]:
tz.tokenize("Noé le dijo a Dios")

['no', '##e', 'le', 'di', '##jo', 'a', 'di', '##os']

In [13]:
input_ids

tensor([[ 101, 2421,  112,  188, 3858, 1996, 3776,  106,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])